<a href="https://colab.research.google.com/github/Ohsoo46/AIFFEL_QUEST/blob/main/Statistics_day05_28_1_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 전처리
df = df.drop(columns=['number', 'name'])
df['legendary'] = df['legendary'].astype(int)

num_cols = ['total','hp','attack','defense','sp_attack','sp_defense','speed','generation']
cat_cols = df.select_dtypes(include = ['object', 'string'])

df['type2'] = df['type2'].fillna('Unknown')
df['type2'] = df['type2'].apply(lambda x: 0 if x == 'Unknown' else 1)




# split
X = df.drop('legendary', axis = 1)
y = df['legendary'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 범주 비교
a = set(X_train['type1'].unique())
b = set(X_test['type1'].unique())
print(b - a)

# scaling
Scaler = MinMaxScaler()
X_train[num_cols] = Scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = Scaler.transform(X_test[num_cols])

# ohe
X_train = pd.get_dummies(X_train, columns = ['type1'], drop_first=True)
X_test = pd.get_dummies(X_test, columns = ['type1'], drop_first=True)

X_test = X_test.reindex(columns = X_train.columns, fill_value=0)

# model

models = {
    'LR' : LogisticRegression(max_iter=1000,random_state=42),
    'DT' : DecisionTreeClassifier(random_state=42),
    'RF' : RandomForestClassifier(n_estimators=200, random_state=42)
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results.append(
        [
            name,
            accuracy_score(y_test, y_pred),
            precision_score(y_test, y_pred),
            recall_score(y_test, y_pred),
            f1_score(y_test, y_pred)
        ]
    )

pd.DataFrame(results, columns = ['Model', 'Acc', 'Pre', 'Recall', 'f1'])

